In [ ]:
# Model Evaluation

# Plot Training History
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Accuracy
axes[0, 0].plot(history.history['accuracy'], label='Train')
axes[0, 0].plot(history.history['val_accuracy'], label='Validation')
axes[0, 0].set_title('Model Accuracy', fontsize=12, fontweight='bold')
axes[0, 0].set_xlabel('Epoch')
axes[0, 0].set_ylabel('Accuracy')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Loss
axes[0, 1].plot(history.history['loss'], label='Train')
axes[0, 1].plot(history.history['val_loss'], label='Validation')
axes[0, 1].set_title('Model Loss', fontsize=12, fontweight='bold')
axes[0, 1].set_xlabel('Epoch')
axes[0, 1].set_ylabel('Loss')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Precision
plt.savefig('../outputs/training_history.png', dpi=300, bbox_inches='tight')
plt.show()

# Make Predictions
y_pred_proba = model.predict(X_val)
y_pred = (y_pred_proba > 0.5).astype(int).flatten()

print(f"Predictions shape: {y_pred.shape}")
print(f"Sample predictions: {y_pred[:10]}")

# Calculate Metrics
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, classification_report, roc_auc_score, roc_curve
)

accuracy = accuracy_score(y_val, y_pred)
precision = precision_score(y_val, y_pred)
recall = recall_score(y_val, y_pred)
f1 = f1_score(y_val, y_pred)
auc = roc_auc_score(y_val, y_pred_proba)

print("\nModel Performance Metrics:")
print("=" * 50)
print(f"Accuracy:  {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"F1 Score:  {f1:.4f}")
print(f"AUC-ROC:   {auc:.4f}")
print("=" * 50)

# Classification report
print("\nDetailed Classification Report:")
print(classification_report(y_val, y_pred, target_names=['Normal', 'Abnormal']))

# Confusion Matrix
cm = confusion_matrix(y_val, y_pred)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Confusion matrix (counts)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[0],
            xticklabels=['Normal', 'Abnormal'],
            yticklabels=['Normal', 'Abnormal'])
axes[0].set_title('Confusion Matrix (Counts)', fontsize=12, fontweight='bold')
axes[0].set_xlabel('Predicted')
axes[0].set_ylabel('Actual')

# Confusion matrix (normalized)
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
sns.heatmap(cm_normalized, annot=True, fmt='.2%', cmap='Blues', ax=axes[1],
            xticklabels=['Normal', 'Abnormal'],
            yticklabels=['Normal', 'Abnormal'])
axes[1].set_title('Confusion Matrix (Normalized)', fontsize=12, fontweight='bold')
axes[1].set_xlabel('Predicted')
axes[1].set_ylabel('Actual')

plt.tight_layout()
plt.savefig('../outputs/confusion_matrix.png', dpi=300, bbox_inches='tight')
plt.show()

# ROC Curve
fpr, tpr, thresholds = roc_curve(y_val, y_pred_proba)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Random Classifier')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve', fontsize=12, fontweight='bold')
plt.legend(loc="lower right")
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('../outputs/roc_curve.png', dpi=300, bbox_inches='tight')
plt.show()

# Summary Metrics Table
metrics_summary = pd.DataFrame({
    'Metric': ['Accuracy', 'Precision', 'Recall', 'F1 Score', 'AUC-ROC'],
    'Value': [accuracy, precision, recall, f1, auc]
})

print("\nMetrics Summary Table:")
print(metrics_summary.to_string(index=False))

# Save metrics
Path('../outputs').mkdir(exist_ok=True)
metrics_summary.to_csv('../outputs/metrics_summary.csv', index=False)
print("\nMetrics saved to ../outputs/metrics_summary.csv")

print("\nNotebook execution complete!")
print("All visualizations saved to ../outputs/")
print("Model saved to ../models/cardiac_cnn_model.h5")

## Notebook Roadmap
- Dataset structure overview & sanity checks
- Audio preprocessing experiments (waveform + log-mel)
- Baseline HeartBeat ensemble training runs
- Evaluation with multi-metric reporting & confusion matrix
- Prediction demos and Grad-CAM-style insights
- Feature interpretation: frequency bands, temporal textures, cadence


In [ ]:
from __future__ import annotations

import json
from pathlib import Path
from typing import Dict

import librosa
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from IPython.display import Audio, Image, display

from src.preprocessing import AudioPreprocessor
from src.train import train_pipeline


In [ ]:
DATA_DIR = Path("../data/train")
CLASSES = [
    "normal_heart",
    "murmur",
    "extrasystole",
    "normal_resp",
    "wheeze",
    "crackle",
]


def build_manifest(root: Path) -> pd.DataFrame:
    records = []
    for label in CLASSES:
        class_dir = root / label
        if not class_dir.exists():
            continue
        for audio_path in class_dir.rglob("*.wav"):
            try:
                y, sr = librosa.load(audio_path.as_posix(), sr=None, mono=True)
                duration = len(y) / sr
            except Exception:
                duration = np.nan
                sr = np.nan
            records.append({"path": str(audio_path), "label": label, "duration": duration, "sr": sr})
    return pd.DataFrame(records)

manifest = build_manifest(DATA_DIR)
manifest.head()


### 1. Data Loading and Exploration

In [ ]:
def preview_sample(label: str):
    subset = manifest[manifest.label == label]
    if subset.empty:
        print(f"No samples for {label}.")
        return
    sample_path = Path(subset.sample(1, random_state=0).iloc[0].path)
    y, sr = librosa.load(sample_path.as_posix(), sr=None)
    display(Audio(y, rate=sr))
    plt.figure(figsize=(10, 2))
    plt.plot(np.linspace(0, len(y) / sr, len(y)), y)
    plt.title(f"Waveform preview: {label}")
    plt.show()

# preview_sample("normal_heart")


In [ ]:
preprocessor = AudioPreprocessor(
    target_sample_rate=4000,
    segment_seconds=5.0,
    n_mels=128,
)

if not manifest.empty:
    example_path = Path(manifest.iloc[0].path)
    waveform = preprocessor.load_waveform(example_path)
    mel = preprocessor.to_mel_spectrogram(waveform)

    fig, ax = plt.subplots(1, 2, figsize=(12, 3))
    ax[0].plot(waveform)
    ax[0].set_title("Normalized Waveform")
    sns.heatmap(mel[..., 0], ax=ax[1], cmap="magma")
    ax[1].set_title("Log-Mel Spectrogram")
    plt.tight_layout()
else:
    print("Preprocessing demo awaiting audio files.")


In [ ]:
conf_plot = Path("../outputs/confusion_matrix.png")
if conf_plot.exists():
    display(Image(filename=conf_plot))
else:
    print("Confusion matrix plot pending training run.")


In [ ]:
# from src.prediction import HeartbeatPredictor
# predictor = HeartbeatPredictor(model_path=Path("../models/heartbeat_model.h5"))
# predictor.load()
# predictor.predict(Path("../data/test/normal_heart/example.wav"))


# 4. Create Training Dataset

In [ ]:
# Create Train/Validation Split
train_df_split, val_df_split = loader.create_train_val_split(test_size=0.2, random_state=42)

# Create Small Dataset for Quick Training (REMOVE max_samples for full training)
print("Creating training dataset (using subset for demo)...")
X_train, y_train = loader.create_mel_spectrogram_dataset(
    train_df_split,
    max_samples=200,  # REMOVE THIS LINE for full dataset
    augment=False
)

print("\nCreating validation dataset...")
X_val, y_val = loader.create_mel_spectrogram_dataset(
    val_df_split,
    max_samples=50,  # REMOVE THIS LINE for full dataset
    augment=False
)

print(f"\nDataset shapes:")
print(f"  X_train: {X_train.shape}, y_train: {y_train.shape}")
print(f"  X_val: {X_val.shape}, y_val: {y_val.shape}")

# 5. Model Training

from tensorflow import keras
from tensorflow.keras import layers

def build_cnn_model(input_shape):
    """Build a simple CNN for mel spectrogram classification"""
    model = keras.Sequential([
        layers.Input(shape=input_shape),
        
        # Conv block 1
        layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),
        
        # Conv block 2
        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),
        
        # Conv block 3
        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),
        
        # Dense layers
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        layers.Dense(1, activation='sigmoid')
    ])
    
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=0.001),
        loss='binary_crossentropy',
        metrics=['accuracy', keras.metrics.Precision(), keras.metrics.Recall()]
    )
    
    return model

model = build_cnn_model(X_train.shape[1:])
model.summary()

# Train Model
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=20,
    batch_size=16,
    callbacks=[
        keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True),
        keras.callbacks.ReduceLROnPlateau(patience=3, factor=0.5)
    ],
    verbose=1
)

# Save model
Path('../models').mkdir(exist_ok=True)
model.save('../models/cardiac_cnn_model.h5')
print("\nModel saved to ../models/cardiac_cnn_model.h5")